In [26]:
import csv
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def get_jobs(pages):
    # Open the CSV file once
    with open('jobs.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['job title', 'job link', 'job location', 'salary', 'skills required', 'company name'])
        writer.writeheader()

        
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        driver = webdriver.Chrome(options=chrome_options)

        for page in pages:
            response = requests.get(page)
            soup = BeautifulSoup(response.content, 'html.parser')

            all_jobs = soup.findAll('div', attrs={'class': "css-pkv5jc"})

            for job in all_jobs:
                try:
                    job_title = job.find('h2').find('a').getText()
                    word = 'data'
                    if word not in job_title.lower():
                        continue
                    
                    job_link = job.find('h2').find('a').get('href')
                    job_location = job.find('span', attrs={'class': "css-5wys0k"}).getText()

                    driver.get(job_link)
                    soup2 = BeautifulSoup(driver.page_source, 'html.parser')

                
                    required_skills_elements = soup2.find_all('span', attrs={'class': 'css-tt12j1 e12tgh591'})
                    skills_required = [element.get_text() for element in required_skills_elements] if required_skills_elements else 'N/A'

                    salary_elements = soup2.findAll('span', attrs={'class': 'css-4xky9y'})
                    salary = salary_elements[3].getText() if len(salary_elements) > 3 else 'NOT FOUND'

                    company_name = job.find('div', attrs={'class': 'css-d7j1kk'}).find('a', attrs={'class': 'css-17s97q8'}).getText().split("-")[0].strip()

                    writer.writerow({
                        'job title': job_title, 
                        'job link': job_link, 
                        'job location': job_location, 
                        'salary': salary, 
                        'skills required': skills_required, 
                        'company name': company_name
                    })

                except Exception as e:
                    
                    print(f"Error processing job: {e}")
        
        driver.quit() 


pages = [f"https://wuzzuf.net/search/jobs/?a=navbl&q=data%20science%20jobs&start={i}" for i in range(0, 55)]

get_jobs(pages)


In [32]:
import pandas as pd
jobsss = pd.read_csv('jobs.csv')
jobsss

jobsss.to_csv('data_science_jobs.csv')
